In [11]:
import pandas as pd
import json
import requests
from datetime import datetime, timezone

## Retrieve Earthquake Data

In [12]:
# Define U.S. Geological Survey base URL
usgs_base_url = 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson'

# We will retrieve a list of earthquakes one year at a time from 1995 through 2023.
# This avoids retrieving too many records at one time.
# Create lists of start and end dates so we can loop through them
start_year = 1995
end_year = 2024
start_dates = [f"{year}-01-01" for year in range(start_year, end_year)]
end_dates = [f"{year}-12-31" for year in range(start_year, end_year)]
# start_dates = ["2017-01-01"]
# end_dates = ["2017-06-30"]

# print(start_dates)
# print(end_dates)

In [13]:
# Define minimum and maximum latitude and longitude of the contiguous U.S.
min_latitude = 25.8400
max_latitude = 49.3800
min_longitude = -124.670
max_longitude = -66.9500

In [14]:
# Define minimum magnitude
min_magnitude = 3.0

### Retrieve Earthquake **Summary** Data
We will us the summary data to extract URLs containing detail earthquake data.

In [15]:
# Create location and magnitude filters
location_filter = f"&minlatitude={min_latitude}&minlongitude={min_longitude}&maxlatitude={max_latitude}&maxlongitude={max_longitude}"
magnitude_filter = f"&minmagnitude={min_magnitude}"

# Create an empty list for later to contain the URLs to the detail data
detail_urls = []

# Loop through years
for i in range(len(start_dates)):
    # Create date filter
    date_filter = f"&starttime={start_dates[i]}&endtime={end_dates[i]}"
    # print(start_dates[i])

    # Create URL to retrieve summary data
    summary_url = f"{usgs_base_url}{date_filter}{location_filter}{magnitude_filter}"
    # print(summary_url)

    # Retrieve summary data
    try:
        summary_response = requests.get(summary_url).json()
        # print(json.dumps(summary_response, indent=4))
    except:
        print(f'Error: {summary_response}')
        print(f'Summary URL: {summary_url}')

    # The URL to extract detailed data is contained in 'features'-->'properties'-->'detail'.
    # Since we only want the data from the 'moment-tensor' and the 'focal-mechanism' sections, we
    # only want the URL for earthquake records that contain those sections. In the summary data, we
    # can find a list of the sections included in the detailed data in
    # 'features'-->'properties'-->'types'.
    for feature in summary_response['features']:
        # Extract 'types' with a list of sections included in the detail data
        types = str(feature['properties']['types'])
        
        # If 'moment-tensor' and 'focal-mechanism' sections are contained in detail data extract the 
        # detail data URL
        if ',moment-tensor,' in types and ',focal-mechanism,' in types:
            detail_urls.append(feature['properties']['detail'])

# print(detail_urls)
print(len(detail_urls))

1558


### Retrieve Earthquake **Detail** Data

In [16]:
# Define a list that will contain the data
earthquake_data_list = []

# Loop through each URL in the 'detail_urls' list and retrieve detail data
for url in detail_urls:
    # Retrieve the earthquake details into an earthquake dictionary.
    try:
        # print(url)
        detail_response = requests.get(url).json()
        # print(json.dumps(detail_response, indent=4))

        # Define some abbreviations to save typing later
        properties = detail_response['properties']
        geometry = detail_response['geometry']
        mt_properties = properties['products']['moment-tensor'][0]['properties']
        fm_properties = properties['products']['focal-mechanism'][0]['properties']

        # Define a list of moment tensor columns we need
        mt_keys_list = ['n-axis-azimuth',
                       'n-axis-length',
                       'n-axis-plunge',
                       'p-axis-azimuth',
                       'p-axis-length',
                       'p-axis-plunge',
                       't-axis-azimuth',
                       't-axis-length',
                       't-axis-plunge',
                       'percent-double-couple',
                       'scalar-moment',
                       'tensor-mpp',
                       'tensor-mrp',
                       'tensor-mrr',
                       'tensor-mtp',
                       'tensor-mrt',
                       'tensor-mtt']
        
        # Define a list of focal mechanism columns we need
        fm_keys_list = ['nodal-plane-1-dip',
                        'nodal-plane-1-rake',
                        'nodal-plane-1-strike',
                        'nodal-plane-2-dip',
                        'nodal-plane-2-rake',
                        'nodal-plane-2-strike']
        
        # Make sure we only retrieve complete records with columns we need
        if all(mt_key in list(mt_properties.keys()) for mt_key in mt_keys_list) and \
           all(fm_key in list(fm_properties.keys()) for fm_key in fm_keys_list):
            print(f"Processing Data for Earthquake {detail_response['id']} at {datetime.fromtimestamp(properties['time']/1000, timezone.utc)}")
            
            # Collect the data into a dictionary
            dict_earthquake = {'id': detail_response['id'],
                              'time': datetime.fromtimestamp(properties['time']/1000, timezone.utc),
                              'place': properties['place'],
                              'longitude': geometry['coordinates'][0],
                              'latitude': geometry['coordinates'][1],
                              'depth': geometry['coordinates'][2],
                              'magnitude': properties['mag'],
                              'felt': properties['felt'],
                              'cdi': properties['cdi'],
                              'mmi': properties['mmi'],
                              'significance': properties['sig'],
                              'number_stations': properties['nst'],
                              'min_station_distance': properties['dmin'],
                              'nodal_plane_1_dip': fm_properties['nodal-plane-1-dip'],
                              'nodal_plane_1_rake': fm_properties['nodal-plane-1-rake'],
                              'nodal_plane_1_strike': fm_properties['nodal-plane-1-strike'],
                              'nodal_plane_2_dip': fm_properties['nodal-plane-2-dip'],
                              'nodal_plane_2_rake': fm_properties['nodal-plane-2-rake'],
                              'nodal_plane_2_strike': fm_properties['nodal-plane-2-strike'],
                              'n_axis_azimuth': mt_properties['n-axis-azimuth'],
                              'n_axis_length' : mt_properties['n-axis-length'],
                              'n_axis_plunge': mt_properties['n-axis-plunge'],
                              'p_axis_azimuth': mt_properties['p-axis-azimuth'],
                              'p_axis_length' : mt_properties['p-axis-length'],
                              'p_axis_plunge': mt_properties['p-axis-plunge'],
                              't_axis_azimuth': mt_properties['t-axis-azimuth'],
                              't_axis_length' : mt_properties['t-axis-length'],
                              't_axis_plunge': mt_properties['t-axis-plunge'],
                              'percent_double_couple': mt_properties['percent-double-couple'],
                              'scalar_moment': mt_properties['scalar-moment'],
                              'tensor_mpp': mt_properties['tensor-mpp'],
                              'tensor_mrp': mt_properties['tensor-mrp'],
                              'tensor_mrr': mt_properties['tensor-mrr'],
                              'tensor_mrt': mt_properties['tensor-mrt'],
                              'tensor_mtp': mt_properties['tensor-mtp'],
                              'tensor_mtt': mt_properties['tensor-mtt']
                              }
            # print(dict_earthquake)
            print("Success")

            # Append the earthquake dictionary to the earthquake list
            earthquake_data_list.append(dict_earthquake)
    except:
        print(f'Failure for URL: {url}')

Processing Data for Earthquake nc30092964 at 1995-12-28 18:28:01.230000+00:00
Success
Processing Data for Earthquake nc30092581 at 1995-12-23 05:39:56.650000+00:00
Success
Processing Data for Earthquake nc30092506 at 1995-12-22 09:00:34.560000+00:00
Success
Processing Data for Earthquake nc30091857 at 1995-12-13 06:25:54.110000+00:00
Success
Processing Data for Earthquake nc30094697 at 1995-12-13 05:45:12.760000+00:00
Success
Processing Data for Earthquake nc30091039 at 1995-12-01 23:11:29.010000+00:00
Success
Processing Data for Earthquake nc30090903 at 1995-11-29 23:12:33.630000+00:00
Success
Processing Data for Earthquake nc30090872 at 1995-11-29 18:47:41.360000+00:00
Success
Processing Data for Earthquake nc30089843 at 1995-11-15 20:33:59.720000+00:00
Success
Processing Data for Earthquake nc30089625 at 1995-11-12 20:59:00.200000+00:00
Success
Processing Data for Earthquake nc30089562 at 1995-11-11 20:19:23.390000+00:00
Success
Processing Data for Earthquake nc30085627 at 1995-09-2

In [17]:
# Create a Pandas DataFrame with earthquake data
df_earthquake_data = pd.DataFrame(earthquake_data_list)

display(df_earthquake_data)

,id,time,place,longitude,latitude,depth,magnitude,felt,cdi,mmi,...,t_axis_length,t_axis_plunge,percent_double_couple,scalar_moment,tensor_mpp,tensor_mrp,tensor_mrr,tensor_mrt,tensor_mtp,tensor_mtt
0,nc30092964,1995-12-28 18:28:01.230000+00:00,"9 km WNW of Topaz Lake, Nevada",-119.654500,38.714500,-1.011,4.80,NaN,NaN,6.100,...,1.780E+16,8.496,0.94,1.749E+16,1.717E+16,-2.593E+15,-2.563E+14,-1.434E+15,2.590E+15,-1.691E+16
1,nc30092581,1995-12-23 05:39:56.650000+00:00,"8 km WNW of Topaz Lake, Nevada",-119.633000,38.730500,-1.081,4.70,NaN,NaN,NaN,...,1.120E+16,8.968,0.83,1.175E+16,1.082E+16,1.325E+15,-2.496E+14,4.455E+15,1.729E+15,-1.057E+16
2,nc30092506,1995-12-22 09:00:34.560000+00:00,California-Nevada border region,-119.635000,38.721500,3.659,4.86,NaN,NaN,NaN,...,2.764E+16,3.308,0.53,2.435E+16,2.737E+16,-2.897E+15,-1.909E+16,-4.715E+15,-1.774E+15,-8.282E+15
3,nc30091857,1995-12-13 06:25:54.110000+00:00,"9 km ESE of Gilroy, California",-121.470333,36.982167,4.234,3.80,NaN,NaN,NaN,...,7.785E+14,15.899,0.81,7.420E+14,4.317E+14,2.781E+14,-4.776E+13,-3.905E+13,5.545E+14,-3.840E+14
4,nc30094697,1995-12-13 05:45:12.760000+00:00,"9 km ESE of Gilroy, California",-121.470000,36.976667,6.204,3.90,NaN,NaN,NaN,...,8.142E+14,9.016,0.49,9.521E+14,3.951E+14,2.489E+13,2.802E+14,1.330E+14,7.743E+14,-6.753E+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,nc73834131,2023-01-19 20:30:07.050000+00:00,"16km WSW of Weitchpec, CA",-123.876500,41.128667,26.260,3.66,17.0,3.4,2.318,...,3.729E+14,5.457,0.94,3.783E+14,3.536E+14,-4.656E+13,-2.680E+12,-7.624E+13,-1.057E+14,-3.509E+14
1424,nc73834041,2023-01-19 17:15:55.930000+00:00,"4km S of San Juan Bautista, CA",-121.531000,36.809667,5.320,3.56,218.0,4.3,4.110,...,2.991E+14,2.794,0.68,2.754E+14,2.980E+14,1.766E+13,-5.190E+13,-3.274E+13,1.110E+13,-2.460E+14
1425,nc73830791,2023-01-10 18:32:14.700000+00:00,"6km E of McKinleyville, CA",-124.030333,40.948833,22.200,3.61,726.0,3.6,3.233,...,3.054E+14,3.768,0.76,3.259E+14,2.037E+14,-6.275E+13,-2.460E+14,1.610E+14,1.605E+14,4.230E+13
1426,nc73827571,2023-01-01 18:35:04.510000+00:00,"15km SE of Rio Dell, CA",-123.971000,40.409000,30.630,5.35,976.0,7.2,6.853,...,1.387E+17,10.172,0.79,1.314E+17,1.220E+17,-2.950E+16,-1.183E+16,-9.049E+15,-5.387E+16,-1.102E+17


## Retrieve Soil Bulk Density Data
We downloaded a file called `wosis_latest_bdwsod.csv` from https://data.isric.org/geonetwork/srv/eng/catalog.search#/metadata/2f99e111-183c-11e9-aba8-a0481ca9e724.  
It is located in the `Resources` folder.

In [18]:
# Import the 'wosis_latest_bdwsod.csv' file into a Pandas DataFrame
df = pd.read_csv('Resources/wosis_latest_bdwsod.csv')
df.head()

,X,Y,profile_id,layer_id,profile_code,layer_name,upper_depth,lower_depth,organic_surface,value,method_options,value_avg,dataset_id,country_name,positional_uncertainty,region,continent,date,licence
0,-64.815834,17.725555,1082594,822988,92P0465,ABss,23,48,0,{1.84},"{""measurement condition = [ oven dry (∼ 105-11...",1.84,US-NCSS,United States Virgin Islands,Circa 100 m,Caribbean,Northern America,1992-3-6,U.S. Public Domain http://www.usa.gov/publicdo...
1,-73.885750,40.604252,1086785,861263,99P0563,C2,85,122,0,{2.12},"{""measurement condition = [ oven dry (∼ 105-11...",2.12,US-NCSS,United States of America,Circa 100 m,Northern America,Northern America,1999-8-8,U.S. Public Domain http://www.usa.gov/publicdo...
2,-96.072220,39.946945,1084215,836095,94P0413,Bss5,155,215,0,{1.81},"{""measurement condition = [ oven dry (∼ 105-11...",1.81,US-NCSS,United States of America,Circa 100 m,Northern America,Northern America,1994-4-18,U.S. Public Domain http://www.usa.gov/publicdo...
3,-90.516670,39.893890,1074644,766974,83P0241,Bw3,79,99,0,{1.48},"{""measurement condition = [ oven dry (∼ 105-11...",1.48,US-NCSS,United States of America,Circa 100 m,Northern America,Northern America,1981-6-1,U.S. Public Domain http://www.usa.gov/publicdo...
4,107.400002,-6.433333,1052485,602983,87P0486,Bt32,127,160,0,{1.42},"{""measurement condition = [ oven dry (∼ 105-11...",1.42,US-NCSS,Indonesia,Over 10 km,South-Eastern Asia,Asia,1987-5-1,U.S. Public Domain http://www.usa.gov/publicdo...


### Clean Soil Bulk Density Data
Note that the data is for the whole earth. So, first we want to restrict the data to the contiguous United States. We will use the same minimum and maximum longitude and latitude we defined above to geographically select the earthquake data. 
We also want to keep only the features we need. The features we need are
* `X` and `Y` corresponding to longitude and latitude
* `upper_depth` and `lower_depth` denoting the upper and lower boundaries of different soil layers
* `value_avg` which contains the soil bulk density

In [19]:
# Select only records corresponding to the contiguous US by using the minimum and maximum 
# longitude and latitude defined above
df_usa = df.loc[(df['X']>=min_longitude) & (df['X']<=max_longitude) &
                (df['Y']>=min_latitude) & (df['Y']<=max_latitude)]

In [20]:
# Define the features to keep
columns_to_keep = ['X', 'Y', 'upper_depth', 'lower_depth', 'value_avg']

# Select those features/columns
df_soil = df_usa[columns_to_keep].sort_values(by=['X', 'Y', 'upper_depth']).reset_index(drop=True)
df_soil.head(10)

,X,Y,upper_depth,lower_depth,value_avg
0,-124.378891,43.235554,0,15,1.22
1,-124.378891,43.235554,31,58,1.41
2,-124.378891,43.235554,84,119,1.38
3,-124.377853,42.926193,0,20,0.96
4,-124.377853,42.926193,20,43,1.07
5,-124.377853,42.926193,43,69,1.37
6,-124.377853,42.926193,69,91,1.47
7,-124.377853,42.926193,91,122,1.55
8,-124.377853,42.926193,122,152,1.65
9,-124.377502,43.188057,18,41,1.25


As noted above, many locations have multiple entries for the soiol bulk density (`value_avg`) depending on the soil layer. We are only interested in one value. Hence, we will average over the different layers.

In [21]:
# Average over all records with the same coordinates
df_soil_bd = df_soil[['X', 'Y', 'value_avg']].groupby(['X', 'Y']).mean().reset_index()

#Rename 'X' and 'Y' columns to 'longitude' and 'latitude', respectively
df_soil_bd.rename(columns={'X': 'longitude', 'Y': 'latitude'}, inplace=True)

df_soil_bd.head()

,longitude,latitude,value_avg
0,-124.378891,43.235554,1.336667
1,-124.377853,42.926193,1.345000
2,-124.377502,43.188057,1.433333
3,-124.368833,40.469861,1.856667
4,-124.368332,43.182220,1.508000


## Combine Earthquake and Soil Density Data

Since the locations in the earthquake data are not identical to the locations in the soil density data we will associate a record in the earthquake data with a record for the closest location in the soil density data.

In [22]:
# Define a function that calculates a measure for the distance between a point (x1, y1) and
# (x2, y2)
def dist(x1, y1, x2, y2):
    return ((x1-x2)**2 + (y1-y2)**2)

In [27]:
soil_densities = []

for idx in df_earthquake_data.index:
    long = df_earthquake_data['longitude'][idx]
    lat = df_earthquake_data['latitude'][idx]

    distance = [dist(df_soil_bd['longitude'][i], df_soil_bd['latitude'][i], long, lat)
                        for i in df_soil_bd.index]
    
    soil_densities.append(df_soil_bd['value_avg'][distance.index(min(distance))])

df_earthquake_data['soil_density'] = soil_densities

df_earthquake_data.head()

,id,time,place,longitude,latitude,depth,magnitude,felt,cdi,mmi,...,t_axis_plunge,percent_double_couple,scalar_moment,tensor_mpp,tensor_mrp,tensor_mrr,tensor_mrt,tensor_mtp,tensor_mtt,soil_density
0,nc30092964,1995-12-28 18:28:01.230000+00:00,"9 km WNW of Topaz Lake, Nevada",-119.654500,38.714500,-1.011,4.80,NaN,NaN,6.1,...,8.496,0.94,1.749E+16,1.717E+16,-2.593E+15,-2.563E+14,-1.434E+15,2.590E+15,-1.691E+16,1.25000
1,nc30092581,1995-12-23 05:39:56.650000+00:00,"8 km WNW of Topaz Lake, Nevada",-119.633000,38.730500,-1.081,4.70,NaN,NaN,NaN,...,8.968,0.83,1.175E+16,1.082E+16,1.325E+15,-2.496E+14,4.455E+15,1.729E+15,-1.057E+16,1.25000
2,nc30092506,1995-12-22 09:00:34.560000+00:00,California-Nevada border region,-119.635000,38.721500,3.659,4.86,NaN,NaN,NaN,...,3.308,0.53,2.435E+16,2.737E+16,-2.897E+15,-1.909E+16,-4.715E+15,-1.774E+15,-8.282E+15,1.25000
3,nc30091857,1995-12-13 06:25:54.110000+00:00,"9 km ESE of Gilroy, California",-121.470333,36.982167,4.234,3.80,NaN,NaN,NaN,...,15.899,0.81,7.420E+14,4.317E+14,2.781E+14,-4.776E+13,-3.905E+13,5.545E+14,-3.840E+14,1.89375
4,nc30094697,1995-12-13 05:45:12.760000+00:00,"9 km ESE of Gilroy, California",-121.470000,36.976667,6.204,3.90,NaN,NaN,NaN,...,9.016,0.49,9.521E+14,3.951E+14,2.489E+13,2.802E+14,1.330E+14,7.743E+14,-6.753E+14,1.89375


In [28]:
# Export earthquake data to a CSV file called 'earthquake_data.csv'
df_earthquake_data.to_csv('../Resources/earthquake_data.csv', index=False)